In [44]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

# Total features: age, rank, height, hand, length of math, points won, %ace, %bp saved, surface, #sets best of, ind set scores, vegas odds, avg rally length, winners, errors

# 1) Get age, rank, height, R/L hand, length of match, points won, %ace, %bp saved, 1st serve %, 1st serve won %, 2nd serve won %

In [19]:
atp_match_year_files = ['atp_matches_2010.csv','atp_matches_2011.csv','atp_matches_2012.csv','atp_matches_2013.csv','atp_matches_2014.csv','atp_matches_2015.csv','atp_matches_2016.csv','atp_matches_2017.csv','atp_matches_2018.csv']

In [20]:
df_years = pd.DataFrame()

In [21]:
for x in atp_match_year_files:
    df = pd.read_csv(x)
    dff = df[['tourney_name','tourney_date','winner_name', 'winner_rank', 'winner_hand', 'winner_ht', 'winner_age', 'loser_name', 'loser_rank', 'loser_hand', 'loser_ht', 'loser_age', 'surface', 'best_of', 'round','minutes','w_ace','w_svpt','w_1stIn','w_1stWon','w_2ndWon','w_bpSaved','w_bpFaced','l_ace','l_svpt','l_1stIn','l_1stWon','l_2ndWon','l_bpSaved','l_bpFaced']]
    dff.rename(columns = {dff.columns[2]:'p1', 'winner_rank':'p1_rank', 'winner_hand':'p1_hand', 'winner_ht':'p1_ht', 'winner_age':'p1_age','w_ace':'p1_ace','w_svpt':'p1_ptswon','w_bpSaved':'p1_bpS','w_bpFaced':'p1_bpF', 'loser_name':'p2', 'loser_rank':'p2_rank', 'loser_hand':'p2_hand', 'loser_ht':'p2_ht', 'loser_age':'p2_age','l_ace':'p2_ace','l_svpt':'p2_ptswon','l_bpSaved':'p2_bpS','l_bpFaced':'p2_bpF', 'surface':'surface', 'best_of':'best_of', 'round':'round'}, inplace=True)
    dff['winner']= df['winner_name']
    
    dff.p1_age = dff.p1_age.round()
    dff.p2_age = dff.p2_age.round()
    
    dff['p1_aceP'] = (dff['p1_ace']/dff['p1_ptswon'])
    dff['p2_aceP'] = (dff['p2_ace']/dff['p2_ptswon'])
    dff['p1_bpP'] = (dff['p1_bpS']/dff['p1_bpF'])
    dff['p2_bpP'] = (dff['p2_bpS']/dff['p2_bpF'])
    
    dff['p1_1st%'] = dff['w_1stIn']/dff['p1_ptswon']
    dff['p2_1st%'] = dff['l_1stIn']/dff['p2_ptswon']
    dff['p1_1stWon%'] = dff['w_1stWon']/dff['w_1stIn']
    dff['p2_1stWon%'] = dff['l_1stWon']/dff['l_1stIn']
    dff['p1_2ndWon%'] = dff['w_2ndWon']/(dff['p1_ptswon']-dff['w_1stIn'])
    dff['p2_2ndWon%'] = dff['l_2ndWon']/(dff['p2_ptswon']-dff['l_1stIn'])
    
    dff['tourney_name'] = dff['tourney_name'].str.replace(' ','_')
    dff['year'] = dff['tourney_date'].apply(str).str[:4]
    dff['p1_fn'], dff['p1_ln'] = dff['p1'].str.split(' ',1).str
    dff['p2_fn'], dff['p2_ln'] = dff['p2'].str.split(' ',1).str
    dff['match_id'] = dff['tourney_name'].astype(str)+'-'+dff['year']+'-'+dff['p1_ln']+'-'+dff['p2_ln']
    
    dff = dff.drop(['tourney_name','tourney_date','p1_ace','w_1stIn','w_1stWon','w_2ndWon','p2_ace','l_1stIn','l_1stWon','l_2ndWon','p1_bpS','p2_bpS','p1_bpF','p2_bpF','p1_fn','p2_fn','year','p1_fn','p1_ln','p2_fn','p2_ln'], axis=1)
    
    df_years =df_years.append(dff)

/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-ver

In [22]:
df_years.shape

(26315, 28)

In [23]:
df_years.head()

,p1,p1_rank,p1_hand,p1_ht,p1_age,p2,p2_rank,p2_hand,p2_ht,p2_age,...,p2_aceP,p1_bpP,p2_bpP,p1_1st%,p2_1st%,p1_1stWon%,p2_1stWon%,p1_2ndWon%,p2_2ndWon%,match_id
0,Simon Greul,60.0,R,185.0,29.0,Paolo Lorenzi,86.0,R,183.0,28.0,...,0.000000,0.888889,0.666667,0.543860,0.656250,0.806452,0.523810,0.538462,0.500000,Auckland-2010-Greul-Lorenzi
1,John Isner,33.0,R,206.0,25.0,Guillermo Garcia Lopez,43.0,R,188.0,27.0,...,0.056180,0.600000,0.571429,0.658824,0.629213,0.803571,0.660714,0.586207,0.666667,Auckland-2010-Isner-Garcia Lopez
2,Juan Monaco,30.0,R,185.0,26.0,Horacio Zeballos,48.0,L,188.0,25.0,...,0.075472,1.000000,0.555556,0.636364,0.547170,0.892857,0.620690,0.500000,0.333333,Auckland-2010-Monaco-Zeballos
3,Michael Lammer,209.0,R,185.0,28.0,Daniel Evans,266.0,R,175.0,20.0,...,0.074627,0.636364,0.200000,0.571429,0.522388,0.607143,0.600000,0.476190,0.562500,Auckland-2010-Lammer-Evans
4,Jose Rubin Statham,332.0,R,183.0,23.0,Daniel King Turner,263.0,R,190.0,26.0,...,0.133333,0.777778,0.705882,0.657534,0.626667,0.687500,0.617021,0.400000,0.357143,Auckland-2010-Rubin Statham-King Turner


In [24]:
df_years.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26315 entries, 0 to 2493
Data columns (total 28 columns):
p1            26252 non-null object
p1_rank       26062 non-null float64
p1_hand       26236 non-null object
p1_ht         22498 non-null float64
p1_age        26243 non-null float64
p2            26252 non-null object
p2_rank       25770 non-null float64
p2_hand       26209 non-null object
p2_ht         20807 non-null float64
p2_age        26228 non-null float64
surface       26134 non-null object
best_of       26252 non-null float64
round         26252 non-null object
minutes       22859 non-null float64
p1_ptswon     24139 non-null float64
p2_ptswon     24139 non-null float64
winner        26252 non-null object
p1_aceP       24131 non-null float64
p2_aceP       24131 non-null float64
p1_bpP        21667 non-null float64
p2_bpP        24044 non-null float64
p1_1st%       24131 non-null float64
p2_1st%       24131 non-null float64
p1_1stWon%    24131 non-null float64
p2_1stWon% 

# 2) Get surface, #sets best of, ind set scores, vegas odds

In [25]:
tournament_files = ['2010.xls','2011.xls','2012.xls','2013.xlsx','2014.xlsx','2015.xlsx','2016.xlsx','2017.xlsx','2018.xlsx',]

In [26]:
df_tournament = pd.DataFrame()

In [27]:
for x in tournament_files:
    df3 = pd.read_excel(x)
    dff3 = df3[['Tournament','Date','Surface','Round','Best of','Winner','Loser','W1','L1','W2','L2','W3','L3','W4','L4','W5','L5','Wsets','Lsets','B365W', 'B365L', 'EXW', 'EXL', 'LBW', 'LBL', 'PSW','PSL']]
    dff3['year'] = dff3['Date'].apply(str).str[:4]
    dff3['p1_ln'] = dff3['Winner'].str[:-3]
    dff3['p2_ln'] = dff3['Loser'].str[:-3]
    
    dff3[['B365W', 'B365L', 'EXW', 'EXL', 'LBW', 'LBL', 'PSW','PSL']] = dff3[['B365W', 'B365L', 'EXW', 'EXL', 'LBW', 'LBL', 'PSW','PSL']].astype(str)
    dff3[['B365W', 'B365L', 'EXW', 'EXL', 'LBW', 'LBL', 'PSW','PSL']] = dff3[['B365W', 'B365L', 'EXW', 'EXL', 'LBW', 'LBL', 'PSW','PSL']].apply(lambda x: x.str.replace(',',''),axis=1)
    dff3[['B365W', 'B365L', 'EXW', 'EXL', 'LBW', 'LBL', 'PSW','PSL']] = dff3[['B365W', 'B365L', 'EXW', 'EXL', 'LBW', 'LBL', 'PSW','PSL']].astype(float)
    
    dff3['p1_bets_count'] = dff3[['B365W', 'EXW', 'LBW', 'PSW']].count(axis=1)
    dff3['p1_sum_bets'] = dff3['B365W'].fillna(0)+dff3['EXW'].fillna(0)+dff3['LBW'].fillna(0)+dff3['PSW'].fillna(0)
    dff3['p1_AvgW'] = dff3['p1_sum_bets']/dff3['p1_bets_count']
    
    dff3['p2_bets_count'] = dff3[['B365L', 'EXL', 'LBL', 'PSL']].count(axis=1) 
    dff3['p2_sum_bets'] = dff3['B365L'].fillna(0)+dff3['EXL'].fillna(0)+dff3['LBL'].fillna(0)+dff3['PSL'].fillna(0)
    dff3['p2_AvgL'] = dff3['p2_sum_bets']/dff3['p2_bets_count']
    
    dff3['p1_wP'] = 1/dff3['p1_AvgW']
    dff3['p2_wP'] = 1/dff3['p2_AvgL']
    dff3['p1_Wp'] = (dff3['p1_wP'])/(dff3['p1_wP'] + dff3['p2_wP'])
    dff3['p2_Wp'] = (dff3['p2_wP'])/(dff3['p1_wP'] + dff3['p2_wP'])
    
    dff3['Tournament'] = dff3['Tournament'].str.replace(' ','_')
    dff3['match_id'] = dff3['Tournament'].astype(str) + '-'+dff3['year']+'-'+dff3['p1_ln']+'-'+dff3['p2_ln']
    dff3 = dff3.drop(['Tournament','Date','year','p1_ln','p2_ln','p1_wP','p2_wP','B365W', 'B365L', 'EXW', 'EXL', 'LBW', 'LBL','PSW', 'PSL', 'year', 'p1_ln', 'p2_ln', 'p1_sum_bets', 'p1_bets_count','p1_AvgW','p2_sum_bets','p2_bets_count','p2_AvgL'], axis=1)
    dff3.iloc[:,5:9] = dff3.iloc[:,5:9].astype('float64')
    df_tournament = df_tournament.append(dff3)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-d

In [28]:
df_tournament.iloc[-150:]

,Surface,Round,Best of,Winner,Loser,W1,L1,W2,L2,W3,L3,W4,L4,W5,L5,Wsets,Lsets,p1_Wp,p2_Wp,match_id
2487,Hard,1st Round,3,Ymer E.,Marterer M.,4.0,6.0,7.0,5.0,7.0,5.0,NaN,NaN,NaN,NaN,2.0,1.0,0.456550,0.543450,Stockholm_Open-2018-Ymer-Marterer
2488,Hard,1st Round,3,Chung H.,Fritz T.,6.0,2.0,6.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,0.532206,0.467794,Stockholm_Open-2018-Chung-Fritz
2489,Hard,1st Round,3,Millman J.,Popyrin A.,4.0,6.0,6.0,3.0,6.0,2.0,NaN,NaN,NaN,NaN,2.0,1.0,0.677658,0.322342,Stockholm_Open-2018-Millman-Popyrin
2490,Hard,1st Round,3,Verdasco F.,Ebden M.,6.0,4.0,6.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,0.616286,0.383714,Stockholm_Open-2018-Verdasco-Ebden
2491,Hard,1st Round,3,Sandgren T.,Pouille L.,6.0,4.0,6.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,0.347337,0.652663,Stockholm_Open-2018-Sandgren-Pouille
2492,Hard,1st Round,3,Lacko L.,Daniel T.,6.0,1.0,6.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,0.458793,0.541207,Stockholm_Open-2018-Lacko-Daniel
2493,Hard,1st Round,3,Gulbis E.,Ymer M.,6.0,4.0,6.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,0.658465,0.341535,Stockholm_Open-2018-Gulbis-Ymer
2494,Hard,1st Round,3,Klahn B.,Copil M.,6.0,2.0,3.0,6.0,6.0,4.0,NaN,NaN,NaN,NaN,2.0,1.0,0.417513,0.582487,Stockholm_Open-2018-Klahn-Copil
2495,Hard,1st Round,3,Kohlschreiber P.,De Minaur A.,7.0,6.0,6.0,7.0,6.0,1.0,NaN,NaN,NaN,NaN,2.0,1.0,0.396250,0.603750,Stockholm_Open-2018-Kohlschreiber-De Minaur
2496,Hard,1st Round,3,Otte O.,Zopp J.,6.0,3.0,7.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,0.499346,0.500654,Stockholm_Open-2018-Otte-Zopp


# 3) Get Rally Length and Winners/Unforced Errors for some matches

In [ ]:
points_df = pd.read_csv('charting-m-points.csv',encoding='latin')
match_id_set = list(sorted(set(points_df['match_id'])))

In [ ]:
# # DO NOT RUN THIS CELL, IMPORT IN THE CSV FILE BELOW
# we_df = pd.DataFrame()


# for x in match_id_set:
#     try:
#         we_dict = {}
#         we_dict['match_id'] = x

#         #get down to correct html on page
#         html_page = pd.read_table('http://www.tennisabstract.com/charting/'+str(x)+'.html')
#         html = html_page.iloc[12][0]
#         soup = BeautifulSoup(html, 'html.parser')

#         #get winners for p1
#         p1_w = soup.findAll('td', {'align':'right'})[7].text
#         p1_wfb = p1_w[p1_w.find("(")+1:p1_w.find(")")]
#         p1_fw, p1_bw = p1_wfb.split('/')

#         #get errors for p1
#         p1_e = soup.findAll('td', {'align':'right'})[8].text
#         p1_efb = p1_e[p1_e.find("(")+1:p1_e.find(")")]
#         p1_fe, p1_be = p1_efb.split('/')

#         p1_winners = int(p1_fw) + int(p1_bw)
#         p1_unforced_errors = int(p1_fe) + int(p1_be)


#         #append to we_dict
#         we_dict['p1_winners'] = p1_winners
#         we_dict['p1_fw'] = p1_fw
#         we_dict['p1_bw'] = p1_bw
#         we_dict['p1_unforced_errors'] = p1_unforced_errors
#         we_dict['p1_fe'] = p1_fe
#         we_dict['p1_be'] = p1_be

#         #get winners for p2
#         p2_w = soup.findAll('td', {'align':'right'})[16].text
#         p2_wfb = p2_w[p2_w.find("(")+1:p2_w.find(")")]
#         p2_fw, p2_bw = p2_wfb.split('/')

#         #get errors for p2
#         p2_e = soup.findAll('td', {'align':'right'})[17].text
#         p2_efb = p2_e[p2_e.find("(")+1:p2_e.find(")")]
#         p2_fe, p2_be = p2_efb.split('/')

#         p2_winners = int(p2_fw) + int(p2_bw)
#         p2_unforced_errors = int(p2_fe) + int(p2_be)

#         #append to we_dict
#         we_dict['p2_winners'] = p2_winners
#         we_dict['p2_fw'] = p2_fw
#         we_dict['p2_bw'] = p2_bw
#         we_dict['p2_unforced_errors'] = p2_unforced_errors
#         we_dict['p2_fe'] = p2_fe
#         we_dict['p2_be'] = p2_be

#         we_df = we_df.append(we_dict, ignore_index=True)
#     except:
#         pass

In [8]:
df2 = pd.read_csv('charting-m-points.csv',encoding='latin')

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (7,9,51) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
we_df = pd.read_csv('we_df.txt', sep='\t')

In [5]:
we_df['date'], we_df['level'], we_df['tourney'], we_df['round'], we_df['p1'], we_df['p2'] = we_df['match_id'].str.split('-',6).str
we_df2 = we_df

In [6]:
we_df2.head()

,Unnamed: 0,match_id,p1_be,p1_bw,p1_fe,p1_fw,p1_unforced_errors,p1_winners,p2_be,p2_bw,p2_fe,p2_fw,p2_unforced_errors,p2_winners,date,level,tourney,round,p1,p2
0,0,19740714-M-Bastad-F-Bjorn_Borg-Adriano_Panatta,9,23,12,22,21.0,45.0,18,12,21,19,39.0,31.0,19740714,M,Bastad,F,Bjorn_Borg,Adriano_Panatta
1,1,19750101-M-Australian_Open-F-Jimmy_Connors-Joh...,11,29,17,15,28.0,44.0,10,13,8,22,18.0,35.0,19750101,M,Australian_Open,F,Jimmy_Connors,John_Newcombe
2,2,19751219-M-Davis_Cup_World_Group_F-RR-Bjorn_Bo...,9,16,6,10,15.0,26.0,9,4,15,8,24.0,12.0,19751219,M,Davis_Cup_World_Group_F,RR,Bjorn_Borg,Jiri_Hrebec
3,3,19780125-M-Pepsi_Grand_Slam-SF-Brian_Gottfried...,21,4,9,7,30.0,11.0,12,7,7,8,19.0,15.0,19780125,M,Pepsi_Grand_Slam,SF,Brian_Gottfried,Bjorn_Borg
4,4,19780611-M-Roland_Garros-F-Bjorn_Borg-Guillerm...,6,7,12,11,18.0,18.0,31,8,14,12,45.0,20.0,19780611,M,Roland_Garros,F,Bjorn_Borg,Guillermo_Vilas


In [9]:
we_df_ph = df2.groupby('match_id')['rallyLen'].mean()
we_df_ph = we_df_ph.to_frame()

In [10]:
we_df2 = pd.merge(we_df2, we_df_ph, on='match_id', how='inner')

In [11]:
# we_df2 = we_df2.drop(['Avg_Rally'], axis=1)

In [12]:
we_df2['p1'] = we_df2['p1'].str.split('_').str[-1]
we_df2['p2'] = we_df2['p2'].str.split('_').str[-1]
we_df2['date'] = we_df2['date'].str[:4]

In [13]:
we_df2 = we_df2.drop(['match_id'], axis=1)

In [14]:
we_df2['match_id'] = we_df2['tourney'].astype(str) + '-'+we_df2['date']+'-'+we_df2['p1']+'-'+we_df2['p2']

In [15]:
we_df2 = we_df2.drop(['date','level','tourney','round','p1','p2'], axis=1)

In [16]:
we_df2 = we_df2.set_index('match_id')

In [17]:
we_df2.head()

,Unnamed: 0,p1_be,p1_bw,p1_fe,p1_fw,p1_unforced_errors,p1_winners,p2_be,p2_bw,p2_fe,p2_fw,p2_unforced_errors,p2_winners,rallyLen
match_id,,,,,,,,,,,,,,
Bastad-1974-Borg-Panatta,0,9,23,12,22,21.0,45.0,18,12,21,19,39.0,31.0,4.111588
Australian_Open-1975-Connors-Newcombe,1,11,29,17,15,28.0,44.0,10,13,8,22,18.0,35.0,2.429091
Davis_Cup_World_Group_F-1975-Borg-Hrebec,2,9,16,6,10,15.0,26.0,9,4,15,8,24.0,12.0,4.350746
Pepsi_Grand_Slam-1978-Gottfried-Borg,3,21,4,9,7,30.0,11.0,12,7,7,8,19.0,15.0,5.375000
Roland_Garros-1978-Borg-Vilas,4,6,7,12,11,18.0,18.0,31,8,14,12,45.0,20.0,9.323308


In [18]:
we_df2.shape

(1362, 14)

# Getting more forced/unforced errors but no rally length

In [45]:
html_page1 = requests.get('http://tennisabstract.com/charting/meta.html')
soup1 = BeautifulSoup(html_page1.content, 'html.parser')

In [46]:
match_id_list2 = soup1.select('a[href*=2]')[2:4005]

In [47]:
match_href_list = []
for x in match_id_list2:
    match_href_list.append(x['href'])

In [ ]:
# we_df_full = pd.DataFrame()

# for x in match_href_list:
#     try:
#         we_dict = {}
#         we_dict['match_id'] = x

#         #get down to correct html on page
#         html_page = pd.read_table('http://www.tennisabstract.com/charting/'+str(x))
#         html = html_page.iloc[12][0]
#         soup = BeautifulSoup(html, 'html.parser')

#         #get winners for p1
#         p1_w = soup.findAll('td', {'align':'right'})[7].text
#         p1_wfb = p1_w[p1_w.find("(")+1:p1_w.find(")")]
#         p1_fw, p1_bw = p1_wfb.split('/')

#         #get errors for p1
#         p1_e = soup.findAll('td', {'align':'right'})[8].text
#         p1_efb = p1_e[p1_e.find("(")+1:p1_e.find(")")]
#         p1_fe, p1_be = p1_efb.split('/')

#         p1_winners = int(p1_fw) + int(p1_bw)
#         p1_unforced_errors = int(p1_fe) + int(p1_be)


#         #append to we_dict
#         we_dict['p1_winners'] = p1_winners
#         we_dict['p1_fw'] = p1_fw
#         we_dict['p1_bw'] = p1_bw
#         we_dict['p1_unforced_errors'] = p1_unforced_errors
#         we_dict['p1_fe'] = p1_fe
#         we_dict['p1_be'] = p1_be

#         #get winners for p2
#         p2_w = soup.findAll('td', {'align':'right'})[16].text
#         p2_wfb = p2_w[p2_w.find("(")+1:p2_w.find(")")]
#         p2_fw, p2_bw = p2_wfb.split('/')

#         #get errors for p2
#         p2_e = soup.findAll('td', {'align':'right'})[17].text
#         p2_efb = p2_e[p2_e.find("(")+1:p2_e.find(")")]
#         p2_fe, p2_be = p2_efb.split('/')

#         p2_winners = int(p2_fw) + int(p2_bw)
#         p2_unforced_errors = int(p2_fe) + int(p2_be)

#         #append to we_dict
#         we_dict['p2_winners'] = p2_winners
#         we_dict['p2_fw'] = p2_fw
#         we_dict['p2_bw'] = p2_bw
#         we_dict['p2_unforced_errors'] = p2_unforced_errors
#         we_dict['p2_fe'] = p2_fe
#         we_dict['p2_be'] = p2_be

#         we_df_full = we_df_full.append(we_dict, ignore_index=True)
#     except:
#         pass

In [29]:
we_df_full = pd.read_csv('we_df_full.txt', sep='\t')

In [30]:
we_df_full = we_df_full.drop(['Unnamed: 0'], axis=1) # only need to run this line if importing pd.read_csv we_df_full.txt
we_df_full['date'], we_df_full['level'], we_df_full['tourney'], we_df_full['round'], we_df_full['p1'], we_df_full['p2'] = we_df_full['match_id'].str.split('-',6).str
we_df_full['p1'] = we_df_full['p1'].str.split('_').str[-1]
we_df_full['p2'] = we_df_full['p2'].str.split('_').str[-1]
we_df_full['date'] = we_df_full['date'].str[:4]

In [31]:
we_df_full['p2'] = we_df_full['p2'].str.split('.').str[-2]
we_df_full = we_df_full.drop(['match_id'], axis=1)
we_df_full['match_id'] = we_df_full['tourney'].astype(str) + '-'+we_df_full['date']+'-'+we_df_full['p1']+'-'+we_df_full['p2']
we_df_full = we_df_full.drop(['date','level','tourney','round','p1','p2'], axis=1)

In [32]:
we_df_full.head()

,p1_be,p1_bw,p1_fe,p1_fw,p1_unforced_errors,p1_winners,p2_be,p2_bw,p2_fe,p2_fw,p2_unforced_errors,p2_winners,match_id
0,8,1,18,1,26.0,2.0,9,4,9,11,18.0,15.0,Australian_Open-2005-Djokovic-Safin
1,16,12,16,8,32.0,20.0,7,11,23,12,30.0,23.0,Monte_Carlo_Masters-2006-Djokovic-Federer
2,16,7,7,13,23.0,20.0,4,1,8,13,12.0,14.0,Roland_Garros-2006-Djokovic-Nadal
3,12,5,14,10,26.0,15.0,7,3,14,13,21.0,16.0,Amersfoort-2006-Djokovic-Massu
4,9,9,15,24,24.0,33.0,14,4,10,8,24.0,12.0,Davis_Cup_World_Group_PO-2006-Federer-Djokovic


In [33]:
we_df_full.shape

(3678, 13)

# 4) Get net point statistics

In [34]:
df5 = pd.read_csv('charting-m-stats-NetPoints.csv',  usecols=(0,1,2,3,4,5,6,7,8,9,10))

In [35]:
df5_N = df5[df5['row'] == 'NetPoints']
df5_N['date'], df5_N['draw'], df5_N['tourney'], df5_N['round'], df5_N['p1'], df5_N['p2']  = df5_N['match_id'].str.split('-',6).str
df5_N['year'] = df5_N['date'].str[:4]
df5_N['p1'] = df5_N['p1'].str.split('_').str[-1]
df5_N['p2'] = df5_N['p2'].str.split('_').str[-1]
df5_N = df5_N.drop(['match_id'], axis=1)
df5_N['match_id'] = df5_N['tourney'].astype(str) + '-'+df5_N['year']+'-'+df5_N['p1']+'-'+df5_N['p2']
df5_N1 = df5_N[df5_N['player'] == 1]
df5_N2 = df5_N[df5_N['player'] == 2]
df5_N1 = df5_N1.drop(['player','row','net_winner','induced_forced','passing_shot_induced_forced','total_shots','date','draw','tourney','round','year'], axis=1)
df5_N2 = df5_N2.drop(['player','row','net_winner','induced_forced','passing_shot_induced_forced','total_shots','date','draw','tourney','round','year'], axis=1)
df5_N1.columns = 'p1_'+df5_N1.columns
df5_N2.columns = 'p2_'+df5_N2.columns

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [36]:
df5_N3 = pd.merge(df5_N1, df5_N2, left_on = 'p1_match_id', right_on = 'p2_match_id')

In [37]:
df5_N3 = df5_N3.drop(['p1_match_id','p2_p1','p2_p2'], axis=1)
df5_N3 = df5_N3.rename(columns={'p1_p1':'p1', 'p1_p2':'p2','p1_pts_won':'p1_netpts_won','p2_pts_won':'p2_netpts_won'})
df5_N3['p1_netwon%'] = df5_N3['p1_netpts_won']/df5_N3['p1_net_pts']
df5_N3['p2_netwon%'] = df5_N3['p2_netpts_won']/df5_N3['p2_net_pts']
df5_N3['p1_net_errors%'] = df5_N3['p1_net_unforced']/df5_N3['p1_net_pts']
df5_N3['p2_net_errors%'] = df5_N3['p2_net_unforced']/df5_N3['p2_net_pts']
df5_N3['p1_passed%'] = df5_N3['p1_passed_at_net']/df5_N3['p1_net_pts']
df5_N3['p2_passed%'] = df5_N3['p2_passed_at_net']/df5_N3['p2_net_pts']
df5_N3 = df5_N3.drop(['p1_netpts_won','p1_net_unforced','p1_passed_at_net','p2_netpts_won','p2_net_unforced','p2_passed_at_net'], axis=1)

In [38]:
df5_N3.tail()

,p1_net_pts,p1,p2,p2_net_pts,p2_match_id,p1_netwon%,p2_netwon%,p1_net_errors%,p2_net_errors%,p1_passed%,p2_passed%
636,9,Gasquet,Paire,13,Miami_Masters-2016-Gasquet-Paire,0.888889,0.615385,0.000000,0.153846,0.111111,0.153846
637,21,Berdych,Johnson,9,Miami_Masters-2016-Berdych-Johnson,0.666667,0.555556,0.000000,0.222222,0.190476,0.222222
638,7,Troicki,Goffin,11,Miami_Masters-2016-Troicki-Goffin,0.571429,0.909091,0.285714,0.000000,0.142857,0.090909
639,19,Raonic,Kyrgios,8,Miami_Masters-2016-Raonic-Kyrgios,0.526316,1.000000,0.210526,0.000000,0.263158,0.000000
640,18,Agut,Tsonga,38,Miami_Masters-2016-Agut-Tsonga,0.611111,0.736842,0.055556,0.052632,0.277778,0.210526


In [39]:
df5_N3.shape

(641, 11)

# 5) More tennis abstract features: return stats, gamepoints, net points

In [48]:
match_href_list

['20050117-M-Australian_Open-R128-Novak_Djokovic-Marat_Safin.html',
 '20060419-M-Monte_Carlo_Masters-R64-Novak_Djokovic-Roger_Federer.html',
 '20060607-M-Roland_Garros-QF-Novak_Djokovic-Rafael_Nadal.html',
 '20060723-M-Amersfoort-F-Novak_Djokovic-Nicolas_Massu.html',
 '20060922-M-Davis_Cup_World_Group_PO-RR-Roger_Federer-Novak_Djokovic.html',
 '20061008-M-Metz-F-Novak_Djokovic-Jurgen_Melzer.html',
 '20061019-M-Madrid_Masters-R16-Andy_Murray-Novak_Djokovic.html',
 '20070107-M-Adelaide-F-Chris_Guccione-Novak_Djokovic.html',
 '20070122-M-Australian_Open-R16-Roger_Federer-Novak_Djokovic.html',
 '20070301-M-Dubai-QF-Roger_Federer-Novak_Djokovic.html',
 '20070305-M-Indian_Wells-SF-Andy_Murray-Novak_Djokovic.html',
 '20070314-M-Indian_Wells_Masters-F-Novak_Djokovic-Rafael_Nadal.html',
 '20070325-M-Miami_Masters-F-Novak_Djokovic-Guillermo_Canas.html',
 '20070329-M-Miami_Masters-QF-Novak_Djokovic-Rafael_Nadal.html',
 '20070330-M-Miami_Masters-SF-Novak_Djokovic-Andy_Murray.html',
 '20070511-M-Ro

In [49]:
tennis_abstract_df = pd.DataFrame()

for x in match_href_list:
    try:
        ta_dict = {}
        ta_dict['match_id'] = x
        
################################
################################
################################

        html_page = pd.read_table('http://www.tennisabstract.com/charting/'+str(x))
        html = html_page.iloc[10][0]
        soup = BeautifulSoup(html, 'html.parser')

        #get break points saved percentage for p1
        p1_bp = soup.findAll('td', {'align':'right'})[1].text
        p1_bp2 = p1_bp[p1_bp.find("(")+1:p1_bp.find(")")]
        p1_bp_saved = float(p1_bp2.strip('%')) / 100.0
        
        #get break points saved percentage for p2
        p2_bp = soup.findAll('td', {'align':'right'})[10].text
        p2_bp2 = p2_bp[p2_bp.find("(")+1:p2_bp.find(")")]
        p2_bp_saved = float(p2_bp2.strip('%')) / 100.0
        
        #p1 game points won
        p1_gp = soup.findAll('td', {'align':'right'})[85].text
        p1_gp2 = p1_gp[p1_gp.find("(")+1:p1_gp.find(")")]
        p1_gp_won = float(p1_gp2.strip('%')) / 100.0
        
        #p2 game points won
        p2_gp = soup.findAll('td', {'align':'right'})[94].text
        p2_gp2 = p2_gp[p2_gp.find("(")+1:p2_gp.find(")")]
        p2_gp_won = float(p2_gp2.strip('%')) / 100.0
        
################################       
################################
################################

        html2 = html_page.iloc[8][0]
        soup2 = BeautifulSoup(html2, 'html.parser')
        
        #p1 total return points faced
        p1_num_rtpts = int(soup2.findAll('td', {'align':'right'})[0].text)
        
        #p1 return points won
        p1_rt = soup2.findAll('td', {'align':'right'})[1].text
        p1_rt2 = p1_rt[p1_rt.find("(")+1:p1_rt.find(")")]
        p1_ret_won = float(p1_rt2.strip('%')) / 100.0
        
        
        #p1 return points in play
        p1_rtip = soup2.findAll('td', {'align':'right'})[4].text
        p1_rtip2 = int(p1_rtip.split(' ')[0])
        p1_ret_inplay = p1_rtip2/p1_num_rtpts
        
        p1_ret_avg_rally = float(soup2.findAll('td', {'align':'right'})[7].text)
        p1_total_shots = p1_ret_avg_rally * p1_num_rtpts

################################      
################################
################################       

        html3 = html_page.iloc[9][0]
        soup3 = BeautifulSoup(html3, 'html.parser')
        
        
        #p2 total return points faced
        p2_num_rtpts = int(soup3.findAll('td', {'align':'right'})[0].text)
        
        #p2 return points won
        p2_rt = soup3.findAll('td', {'align':'right'})[1].text
        p2_rt2 = p2_rt[p2_rt.find("(")+1:p2_rt.find(")")]
        p2_ret_won = float(p2_rt2.strip('%')) / 100.0
        
        
        #p2 return points in play
        p2_rtip = soup3.findAll('td', {'align':'right'})[4].text
        p2_rtip2 = int(p2_rtip.split(' ')[0])
        p2_ret_inplay = p2_rtip2/p2_num_rtpts
        
        p2_ret_avg_rally = float(soup3.findAll('td', {'align':'right'})[7].text)
        p2_total_shots = p2_ret_avg_rally * p2_num_rtpts
        
        #average rally length
        avg_rally_len = (p1_total_shots + p2_total_shots)/(p2_num_rtpts + p1_num_rtpts)

################################
################################
################################
        
        html4 = html_page.iloc[17][0]
        soup4 = BeautifulSoup(html4, 'html.parser')
        
        #p1 total points at net
        p1_netpts = int(soup4.findAll('td', {'align':'right'})[0].text)
        
        #p1 net points won
        p1_np = soup4.findAll('td', {'align':'right'})[1].text
        p1_np2 = p1_np[p1_np.find("(")+1:p1_np.find(")")]
        p1_netwon = float(p1_np2.strip('%')) / 100.0
        
        #p1 net errors
        p1_ne = soup4.findAll('td', {'align':'right'})[4].text
        p1_ne2 = p1_ne[p1_ne.find("(")+1:p1_ne.find(")")]
        p1_neterrors = float(p1_ne2.strip('%')) / 100.0
        
        #p1 passed at net
        p1_pan = soup4.findAll('td', {'align':'right'})[5].text
        p1_pan2 = p1_pan[p1_pan.find("(")+1:p1_pan.find(")")]
        p1_netpassed = float(p1_pan2.strip('%')) / 100.0

################################
################################
################################
             
        html5 = html_page.iloc[18][0]
        soup5 = BeautifulSoup(html5, 'html.parser')
        
        #p2 total points at net
        p2_netpts = int(soup5.findAll('td', {'align':'right'})[0].text)
        
        #p2 net points won
        p2_np = soup5.findAll('td', {'align':'right'})[1].text
        p2_np2 = p2_np[p2_np.find("(")+1:p2_np.find(")")]
        p2_netwon = float(p2_np2.strip('%')) / 100.0
        
        #p2 net errors
        p2_ne = soup5.findAll('td', {'align':'right'})[4].text
        p2_ne2 = p2_ne[p2_ne.find("(")+1:p2_ne.find(")")]
        p2_neterrors = float(p2_ne2.strip('%')) / 100.0
        
        #p2 passed at net
        p2_pan = soup5.findAll('td', {'align':'right'})[5].text
        p2_pan2 = p2_pan[p2_pan.find("(")+1:p2_pan.find(")")]
        p2_netpassed = float(p2_pan2.strip('%')) / 100.0
        
        #p1 points at net percent
        p1_netpts_per = p1_netpts/(p1_num_rtpts + p2_num_rtpts)
        
        #p2 points at net percent
        p2_netpts_per = p2_netpts/(p1_num_rtpts + p2_num_rtpts)

################################
################################
################################
             
        html6 = html_page.iloc[13][0]
        soup6 = BeautifulSoup(html6, 'html.parser')
        
        #p1 total shots
        p1_total_shots = int(soup6.findAll('td', {'align':'right'})[0].text)
        
        #p1 forehand shots percent
        p1_forehand_shots = int(soup6.findAll('td', {'align':'right'})[17].text)
        p1_forehand_per = p1_forehand_shots/p1_total_shots
        
        #p1 backhand shots percent
        p1_backhand_shots = int(soup6.findAll('td', {'align':'right'})[25].text)
        p1_backhand_per = p1_backhand_shots/p1_total_shots
        
################################
################################
################################
           
        html7 = html_page.iloc[14][0]
        soup7 = BeautifulSoup(html7, 'html.parser')
        
        #p2 total shots
        p2_total_shots = int(soup7.findAll('td', {'align':'right'})[0].text)
        
        #p2 forehand shots percent
        p2_forehand_shots = int(soup7.findAll('td', {'align':'right'})[17].text)
        p2_forehand_per = p2_forehand_shots/p2_total_shots
        
        #p2 backhand shots percent
        p2_backhand_shots = int(soup7.findAll('td', {'align':'right'})[25].text)
        p2_backhand_per = p2_backhand_shots/p2_total_shots
        
################################
################################
################################
           
        html8 = html_page.iloc[15][0]
        soup8 = BeautifulSoup(html8, 'html.parser')
        
        #p1 cross court shot percent
        p1_cs = soup8.findAll('td', {'align':'right'})[0].text
        p1_cs2 = p1_cs[p1_cs.find("(")+1:p1_cs.find(")")]
        p1_cc = float(p1_cs2.strip('%')) / 100.0
        
        #p1 down middle shot percent
        p1_dms = soup8.findAll('td', {'align':'right'})[1].text
        p1_dms2 = p1_dms[p1_dms.find("(")+1:p1_dms.find(")")]
        p1_dm = float(p1_dms2.strip('%')) / 100.0
        
        #p1 down line shot percent
        p1_dls = soup8.findAll('td', {'align':'right'})[2].text
        p1_dls2 = p1_dls[p1_dls.find("(")+1:p1_dls.find(")")]
        p1_dl = float(p1_dls2.strip('%')) / 100.0
        
        #p1 inside out shot percent
        p1_ios = soup8.findAll('td', {'align':'right'})[3].text
        p1_ios2 = p1_ios[p1_ios.find("(")+1:p1_ios.find(")")]
        p1_io = float(p1_ios2.strip('%')) / 100.0
        
        #p1 inside in shot percent
        p1_iis = soup8.findAll('td', {'align':'right'})[4].text
        p1_iis2 = p1_iis[p1_iis.find("(")+1:p1_iis.find(")")]
        p1_ii = float(p1_iis2.strip('%')) / 100.0
              
################################
################################
################################
           
        html9 = html_page.iloc[16][0]
        soup9 = BeautifulSoup(html9, 'html.parser')
        
        #p2 cross court shot percent
        p2_cs = soup9.findAll('td', {'align':'right'})[0].text
        p2_cs2 = p2_cs[p2_cs.find("(")+1:p2_cs.find(")")]
        p2_cc = float(p2_cs2.strip('%')) / 100.0
        
        #p2 down middle shot percent
        p2_dms = soup9.findAll('td', {'align':'right'})[1].text
        p2_dms2 = p2_dms[p2_dms.find("(")+1:p2_dms.find(")")]
        p2_dm = float(p2_dms2.strip('%')) / 100.0
        
        #p2 down line shot percent
        p2_dls = soup9.findAll('td', {'align':'right'})[2].text
        p2_dls2 = p2_dls[p2_dls.find("(")+1:p2_dls.find(")")]
        p2_dl = float(p2_dls2.strip('%')) / 100.0
        
        #p2 inside out shot percent
        p2_ios = soup9.findAll('td', {'align':'right'})[3].text
        p2_ios2 = p2_ios[p2_ios.find("(")+1:p2_ios.find(")")]
        p2_io = float(p2_ios2.strip('%')) / 100.0
        
        #p2 inside in shot percent
        p2_iis = soup9.findAll('td', {'align':'right'})[4].text
        p2_iis2 = p2_iis[p2_iis.find("(")+1:p2_iis.find(")")]
        p2_ii = float(p2_iis2.strip('%')) / 100.0
                         
################################
################################
################################

        #append to ta_dict
        ta_dict['p1_bp_saved'] = p1_bp_saved
        ta_dict['p2_bp_saved'] = p2_bp_saved
        ta_dict['p1_gp_won'] = p1_gp_won
        ta_dict['p2_gp_won'] = p2_gp_won
        ta_dict['p1_num_rtpts'] = p1_num_rtpts
        ta_dict['p1_ret_won'] = p1_ret_won
        ta_dict['p1_ret_inplay'] = p1_ret_inplay
        ta_dict['p2_num_rtpts'] = p2_num_rtpts
        ta_dict['p2_ret_won'] = p2_ret_won
        ta_dict['p2_ret_inplay'] = p2_ret_inplay
        ta_dict['avg_rally_len'] = avg_rally_len
        ta_dict['p1_netpts'] = p1_netpts
        ta_dict['p1_netpts_per'] = p1_netpts_per
        ta_dict['p1_netwon'] = p1_netwon
        ta_dict['p1_neterrors'] = p1_neterrors
        ta_dict['p1_netpassed'] = p1_netpassed
        ta_dict['p2_netpts'] = p2_netpts
        ta_dict['p2_netpts_per'] = p2_netpts_per
        ta_dict['p2_netwon'] = p2_netwon
        ta_dict['p2_neterrors'] = p2_neterrors
        ta_dict['p2_netpassed'] = p2_netpassed
        ta_dict['p1_forehand_per'] = p1_forehand_per
        ta_dict['p1_backhand_per'] = p1_backhand_per
        ta_dict['p2_forehand_per'] = p2_forehand_per
        ta_dict['p2_backhand_per'] = p2_backhand_per
        ta_dict['p1_cc'] = p1_cc
        ta_dict['p1_dm'] = p1_dm
        ta_dict['p1_dl'] = p1_dl
        ta_dict['p1_io'] = p1_io
        ta_dict['p1_ii'] = p1_ii
        ta_dict['p2_cc'] = p2_cc
        ta_dict['p2_dm'] = p2_dm
        ta_dict['p2_dl'] = p2_dl
        ta_dict['p2_io'] = p2_io
        ta_dict['p2_ii'] = p2_ii

        tennis_abstract_df = tennis_abstract_df.append(ta_dict, ignore_index=True)
    except:
        pass

In [52]:
tennis_abstract_df.shape

(3219, 36)

In [53]:
tennis_abstract_df.to_csv('tennis_abstract_df.txt', sep='\t')

In [54]:
pd.read_csv('tennis_abstract_df.txt', sep='\t')

,Unnamed: 0,avg_rally_len,match_id,p1_backhand_per,p1_bp_saved,p1_cc,p1_dl,p1_dm,p1_forehand_per,p1_gp_won,...,p2_ii,p2_io,p2_neterrors,p2_netpassed,p2_netpts,p2_netpts_per,p2_netwon,p2_num_rtpts,p2_ret_inplay,p2_ret_won
0,0,4.233333,20050117-M-Australian_Open-R128-Novak_Djokovic...,0.427861,0.31,0.36,0.24,0.27,0.572139,0.36,...,0.01,0.13,0.04,0.00,25.0,0.213675,0.84,66.0,0.833333,0.68
1,1,4.298788,20060419-M-Monte_Carlo_Masters-R64-Novak_Djoko...,0.486486,0.73,0.33,0.08,0.39,0.513514,0.57,...,0.00,0.11,0.28,0.06,18.0,0.109091,0.56,94.0,0.808511,0.41
2,2,5.532710,20060607-M-Roland_Garros-QF-Novak_Djokovic-Raf...,0.511538,0.40,0.36,0.14,0.25,0.488462,0.57,...,0.04,0.14,0.00,0.00,3.0,0.028037,0.67,57.0,0.859649,0.49
3,3,4.300000,20060723-M-Amersfoort-F-Novak_Djokovic-Nicolas...,0.514815,0.50,0.49,0.09,0.23,0.481481,0.79,...,0.03,0.20,0.00,0.10,10.0,0.068966,0.50,67.0,0.626866,0.37
4,4,4.018182,20060922-M-Davis_Cup_World_Group_PO-RR-Roger_F...,0.469799,0.80,0.59,0.17,0.12,0.530201,0.71,...,0.05,0.16,0.06,0.34,35.0,0.198864,0.49,80.0,0.612500,0.31
5,5,3.500000,20061008-M-Metz-F-Novak_Djokovic-Jurgen_Melzer...,0.541322,0.78,0.32,0.17,0.32,0.458678,0.72,...,0.00,0.13,0.09,0.09,35.0,0.197740,0.77,88.0,0.647727,0.35
6,6,5.852571,20061019-M-Madrid_Masters-R16-Andy_Murray-Nova...,0.528384,0.50,0.46,0.05,0.38,0.471616,0.62,...,0.02,0.17,0.05,0.14,22.0,0.125714,0.68,86.0,0.709302,0.37
7,7,2.922018,20070107-M-Adelaide-F-Chris_Guccione-Novak_Djo...,0.637555,0.60,0.32,0.12,0.36,0.362445,0.59,...,0.00,0.19,0.00,0.09,22.0,0.100917,0.77,105.0,0.561905,0.38
8,8,4.113812,20070122-M-Australian_Open-R16-Roger_Federer-N...,0.476190,0.50,0.43,0.14,0.31,0.523810,0.84,...,0.02,0.16,0.13,0.30,23.0,0.127072,0.48,80.0,0.637500,0.25
9,9,3.796954,20070301-M-Dubai-QF-Roger_Federer-Novak_Djokov...,0.541935,0.60,0.39,0.11,0.38,0.458065,0.76,...,0.00,0.16,0.14,0.14,22.0,0.111675,0.55,100.0,0.730000,0.32
